In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import pickle
import joblib
import os

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

In [6]:
def load_dataset(folder_name):


In [8]:
# load dataset
folder_name = 'hot_norm'

with open(f'../data/{folder_name}/train.pkl', 'rb') as f:
    X_train, y_train = pickle.load(f)
with open(f'../data/{folder_name}/val.pkl', 'rb') as f:
    X_val, y_val = pickle.load(f)
with open(f'../data/{folder_name}/test.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

with open(f'../data/scaler_y.pkl', 'rb') as f:
    scaler_y = pickle.load(f)


In [9]:
original = []

# Load model
knn = KNeighborsRegressor(n_neighbors=5)
# Fit model
knn.fit(X_train, y_train)
# Zip data and labels
for x, y, nsplit in zip([X_train, X_val, X_test], [y_train, y_val, y_test], ['train', 'val', 'test']):
    # Make predictions                
    preds = knn.predict(x)
    # Inverse transform normalized predictions
    preds_original = scaler_y.inverse_transform(preds.reshape(-1, 1)).flatten()
    # Inverse transform true values
    y_original = scaler_y.inverse_transform(y.reshape(-1, 1)).flatten()
    # Get MAE
    mae = mean_absolute_error(y_original, preds_original)
    # Append to performances
    original.append({
        'model': 'KNN-5',
        'split': nsplit,
        'mae': mae.round(4),
        'preds': preds_original,
        'y': y_original
    })

In [10]:
# Define the models and splits we are interested in
models_of_interest = ['KNN-5']
splits_of_interest = ['train', 'val', 'test']

# Header for the output
print(f"{'Model':<10} | {'Split':<5} | {'MAE':>5}")

# Loop through each model and split to print the MAE scores
for model in models_of_interest:
    for split in splits_of_interest:
        # Find the relevant performance entry
        entry = next((item for item in original if item['model'] == model and item['split'] == split), None)
        if entry is not None:
            # Print the model, split, and MAE score
            print(f"{model:<10} | {split:<5} | {entry['mae']:>5.4f}")
        else:
            # Handle cases where no entry is found
            print(f"{model:<10} | {split:<5} | {'N/A':>5}")


Model      | Split |   MAE
KNN-5      | train | 55.0942
KNN-5      | val   | 69.2441
KNN-5      | test  | 69.4657
